In [1]:
import numpy as np
import pandas as pd 
from df_name import name

invoice = name(pd.read_csv("C:\\class folder\\capstone\\Final_invoice.csv", low_memory=False))
plant = name(pd.read_excel("C:\\class folder\\capstone\\Plant Master.xlsx"))
cust_data = name(pd.read_excel("C:\\class folder\\capstone\\Customer_Data.xlsx"))
jtd = name(pd.read_csv("C:\\class folder\\capstone\\JTD.csv", low_memory=False))

###################Cleaning Final Invoice###########################

#plant column cleaning 
#cleaning null values
for col in invoice.columns:
    if invoice[col].isnull().sum()/len(invoice)*100 > 50:
        invoice.drop(col, inplace = True, axis = 1)
invoice.isnull().sum()/len(invoice)*100
null = ["area_/_locality","technician_name","model"]
for col in null:
    invoice[col].fillna("unknown", inplace = True)
invoice["area_/_locality"] = invoice["area_/_locality"].map(lambda x:
                                                            "unkown" if x in [",",".","unknown"] else x)
invoice["area_/_locality"] = invoice["area_/_locality"].map(lambda x:
                                                            "Thane" if x == "THNAE" else x)
invoice["area_/_locality"] = invoice["area_/_locality"].map(lambda x: x.title())
invoice.technician_name = invoice["technician_name"].map(lambda x: x.title())
#cleaning state
invoice['district']= invoice['district'].replace(['Andaman and Nico.In.','Arunachal Pradesh','Delhi','Lakshadweep','Daman and Diu','Dadra and Nagar Hav.','Puducherry','Megalaya','Jammu and Kashmir'],
                                         ['Andaman & Nicobar Island','Arunanchal Pradesh','NCT of Delhi','Lakshadweep','Daman & Diu','Dadara & Nagar Havelli','Puducherry','Meghalaya','Jammu & Kashmir']) 
invoice.rename(columns={"district":"state"}, inplace=True)
# cleaning city
invoice.city.fillna("unknown", inplace=True)
invoice.city = invoice.city.map(lambda x: "unknown" if x in ["w",",","."] else x)
invoice.city = invoice.city.map(lambda x: "Kalwa"   if x =="kalawa" else x)                           #kalwa
invoice.city = invoice.city.map(lambda x: "Thane"   if (x.lower().startswith("thane")
                                                        or x.split()[-1] == "thane"
                                                        or x in ["thyane", "THNAE"]) 
                                                    else x)                                           #thane
invoice.city = invoice.city.map(lambda x: "Mumbai" if x in ["mubai","mumabi"] else x)               #mumbai
invoice.city = invoice.city.map(lambda x: "Navi Mumbai" if x in ["navimumbai","new mumbai"] else x) #nav mumbai
invoice.city = invoice.city.map(lambda x: "Pune" if x.split()[-1]=="pune" else x)                   #pune
invoice.city = invoice.city.map(lambda x: "Nashik" if x in ["nasik","naik"] else x)                 #nashik
invoice.city = invoice.city.map(lambda x: "Mulund" if x in ["mulund-w","mulund-e"] else x)          #mulund
#cleaning numerical data
num_col = ["odn_no.","kms_reading","labour_total","misc_total","osl_total",
           "outstanding_amt","parts_total","recovrbl_exp","tds_amount","total_amt_wtd_tax.",
           "total_cgst","total_gst","total_igst","total_sgst/ugst","total_value"]

for col in num_col:
    invoice[col].fillna(invoice[col].median(),inplace=True)

invoice.regn_no.fillna("unknown", inplace=True)

null_col = ["amt_rcvd_from_custom", "amt_rcvd_from_ins_co", "cgst(14%)", "cgst(2.5%)",
            "cgst(6%)", "cgst(9%)", "igst(12%)", "igst(28%)", "igst(18%)", "igst(5%)", 
            "sgst/ugst(14%)", "sgst/ugst(6%)", "sgst/ugst(2.5%)", "sgst/ugst(9%)"]
invoice.drop(null_col, axis = 1, inplace = True)

invoice['customer_no.']=invoice['customer_no.'].str.lstrip("0")

rm_col = ["outstanding_amt", "tds_amount", "total_cgst", "total_gst", "total_igst", "total_sgst/ugst", "recovrbl_exp"]
invoice.drop(rm_col, axis=1, inplace=True)

invoice['customer_no.']=invoice['customer_no.'].str.lstrip("0")

###################Cleaning Plant Master###########################

plant.drop(["vendor_number_plant","plant"], axis=1, inplace=True)
null_col = ["name_2", "house_number_and_street", "po_box", "sales_organization"]
for col in null_col:
    plant[col].fillna("unknown", inplace=True)

#######################Cleaning JTD################################

jtd.dropna(inplace=True)

##################Cleaning Customer Data###########################
cust_data["customer_no."] = cust_data["customer_no."].astype(str)

cust_data['occupation'] = cust_data['occupation'].replace([i for i in range(1,32)],['Director','Sr manager','VP','Asst VP','Sr VP','Col','Lt','Lt Gen','Dr','Capt','Exec Dir','Exec asst','Secretary','Reg Mang','Mang','Jr Mang','DGM','Gen Mang','Sr Gen Mang','Proprietor','Partner','VC','CFO','CTO','Self','MD','Executive','CEO','Customer','VIP Customer','Others'])

cust_data['data_origin']= cust_data['data_origin'].replace(['Z001','Z002','Z003','Z004','Z005','Z006','Z007','Z008','Z009','Z010','Z011','Z012','Z013','Z014','Z015','Z016','Z017','Z018','Z019','Z020','Z021','Z022'],['Campoutdoor','Campworkshop','Emailers','Fleet','Customer','Referncemplyee','Refencused car dealer','Just dialother helpline','Snapdealother website','Company website','Float Activity','Petrol pump activity','Hoardingoutdoor ads','Insurance co.','TV AD','Newspaper AD','newspaper leaflet','Outdoor Sales activity','Spotted outlet','Mahindra sister concern employee','Other outdoor activity','Radio'])

cust_data['partner_Type']=cust_data['partner_type'].replace([1.0,2.0,3.0,9001.0,9003.0,4.0,9002.0],['Retail','Corporate','Fleet','Insuranceco.','ContactPerson','Employee','Surveyor'])

"""
for col in cust_data.columns:
    if cust_data[col].isnull().sum()/len(cust_data)*100 > 50:
        cust_data.drop(col, axis = 1, inplace=True)
"""

cust_data.title.fillna("unkown", inplace = True)

cust_data.dropna(axis=1, inplace=True)

cust_data["customer_no."] = cust_data["customer_no."].astype(str)
##################Saving clean Data###########################

invoice.to_csv("final_invoice_clean.csv")
plant.to_csv("plant_clean.csv")
jtd.to_csv("jtd_clean.csv")
cust_data.to_csv("customer_data_clean.csv")

print("Cleaning Done")    


Cleaning Done


In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import plotly.graph_objs as go 
import dash
import dash_core_components as dcc 
import dash_html_components as html
from dash.dependencies import Input, Output
from df_name import name
import json
from datetime import  datetime as dt
import plotly.express as px
import plotly.io as pio
pio.renderers.default = 'browser'


#importing data
invoice = pd.read_csv("final_invoice_clean.csv")
dm=pd.DataFrame(invoice.groupby(['state'])['model'].count())
dm.reset_index(inplace = True)

invoice.invoice_date = pd.to_datetime(invoice.invoice_date)
invoice["invoice_year"] = invoice["invoice_date"].map(lambda x: x.year)

#data for revenue trends
month_revenue = invoice.groupby(["invoice_year"])["total_value"].sum().reset_index()
revenue_trend_data = go.Line(
    x = month_revenue.invoice_year,
    y = month_revenue.total_value
)


#India map visuialization 
india_states=json.load(open('states_india.geojson', 'r'))
state_id_map = {}
for feature in india_states["features"]:
    feature["id"] = feature["properties"]["state_code"]
    state_id_map[feature["properties"]["st_nm"]] = feature["id"]
dm["id"] = dm["state"].apply(lambda x: state_id_map[x])

fig = px.choropleth(
    dm,
    locations="id",
    geojson=india_states,
    color="model",
    hover_name="state",
    hover_data=["model"],
    title="Car Distribution",
)
fig.update_geos(fitbounds="locations", visible=False)

#data for make wise model
make_model_options = [{"label" : i, "value" : i}
                       for i in invoice.make.unique()]
                    
#data for state dropdown
state_options = [{"label" : i, "value" : i}
                  for i in invoice.state.unique()]

#data for total value for a city
city_options = [{"label":i, "value":i}
            for i in invoice.city.unique()]

#data for total value for a day, selecting date(invoice date)
date_min = invoice.invoice_date.min()
date_max = invoice.invoice_date.max()

#app
app = dash.Dash(__name__)
app.layout = html.Div(
    children = [
        html.H1("Mahindra Data Analysis", style={"text-align":"center"}),
        html.H2("K.J. Somaiya", style={"text-align":"center"}),
        html.Div(
            children = [
                html.H2("State wise distribution of models of vehicals", style={"text-align":"center"}),
                dcc.Graph(
                    figure = fig
                )
            ]
        ),
        html.Div(
            children = [
                html.H2("Make wise Model distribution"),
                dcc.Dropdown(
                    id = "make-dropdown",
                    options = make_model_options,
                    value = "TATA MOTORS"
                ),
                html.Div(id="make-model")
            ]
        ),
        html.Div(
            children = [
                html.H2("State wise Model distribution"),
                dcc.Dropdown(
                    id = "state-dropdown",
                    options = state_options,
                    value = "Maharashtra"
                ),
                html.Div(id="state-model")
            ]
        ),
        html.Div(
            children = [
                html.H2("Revenue trend over time"),
                dcc.Graph(
                    id = "revenue-trend",
                    figure = {
                        "data" : [revenue_trend_data],
                        "layout" : go.Layout(
                            xaxis = {"title" : "Years"}, 
                            yaxis = {"title" : "Total Value(Rs)"}
                        )
                    }
                )
            ]
        ),
         
        html.Div(
            children = [
                html.H2("Select a state to checks it's revenue"),
                dcc.Dropdown(
                    id = "state-rev-dropdown",
                    options = state_options,
                    value = "Maharashtra"
                ),
                html.Div(id="state-revenue")
            ]
        ),
        html.Div(
            children = [
                html.H2("Select a day to check it's revenue"),
                dcc.DatePickerSingle(
                    id = "date-value",
                    min_date_allowed = date_min,
                    max_date_allowed = date_max,
                    date = date_min
                ),
                html.Div(id="value-date")
            ]
        ),
        html.Div(
            children = [
                html.H2("State wise yearly active users"),
                dcc.Dropdown(
                    id = "state-mau",
                    options = state_options,
                    value = "Maharashtra"
                ),
                html.Div(id="mau")
            ]
        )
    ]
)



#make wise model function
@app.callback(
    Output(component_id="make-model", component_property="children"),
    [Input(component_id="make-dropdown", component_property="value")]
)
def make_model(maker):
    df = invoice[invoice.make == maker]
    model_vc = df.model.value_counts()
    model_data = go.Bar(
        x = model_vc.index, 
        y = [model_vc[i] for i in model_vc.index]
    )
    bar = dcc.Graph(
        id = f"{maker}-model",
        figure = {
            "data" : [model_data],
            "layout" : go.Layout(
                xaxis = {"title" : f"Models for {maker}"}, 
                yaxis = {"title" : "Frequency"}
            )
        }
    )
    return bar

#state wise model
@app.callback(
    Output(component_id="state-model", component_property="children"),
    [Input(component_id="state-dropdown", component_property="value")]
)
def make_model(instate):
    df = invoice[invoice.state == instate]
    model_vc = df.model.value_counts()
    model_data = go.Bar(
        x = model_vc.index, 
        y = [model_vc[i] for i in model_vc.index]
    )
    bar = dcc.Graph(
        id = f"{instate}-model",
        figure = {
            "data" : [model_data],
            "layout" : go.Layout(
                xaxis = {"title" : f"Models for {instate}"}, 
                yaxis = {"title" : "Frequency"}
            )
        }
    )
    return bar

#state wise revenue
@app.callback(
    Output(component_id="state-revenue", component_property="children"),
    [Input(component_id="state-rev-dropdown", component_property="value")]
)
def state_revenue(instate):
    df = invoice[invoice.state == instate]
    value = np.round(df.total_value.sum() ,2)
    return f"Total Value for {instate} = Rs. {value}"


#date wise value
@app.callback(
    Output(component_id="value-date", component_property="children"),
    [Input(component_id="date-value", component_property="date")]
)
def value_on_date(date):
    df = invoice[invoice.invoice_date==date]
    df.dropna(inplace=True)
    value = np.round(np.sum(df.total_value),2)
    return f"Total value for {date} = Rs. {value}"

#state wise monthly active users
@app.callback(
    Output(component_id="mau", component_property="children"),
    [Input(component_id="state-mau", component_property="value")]
)
def state_mau(instate):
    df = invoice[invoice.state==instate]
    df_2 = df.groupby("invoice_year")["user_id"].nunique().reset_index()
    X = df_2.invoice_year
    Y = df_2.user_id
    data = go.Line(
        x = X,
        y = Y
    )
    plot = dcc.Graph(
        id = f"{instate}-mau",
        figure = {
            "data" : [data],
            "layout" : go.Layout(
                xaxis = {"title" : "Years"},
                yaxis = {"title" : "Active users"})
        }
    )
    return plot

#state wise revenue plot
# @app.callback(
#     Output(component_id="state-revenue", component_property="children"),
#     [Input(component_id="revenue-state", component_property="value")]
# )
# def state_revenue(instate):
#     df = invoice[invoice.state == instate]
#     X = df.invoice_year
#     Y = df.total_value
#     data = go.Line(
#         x = X,
#         y = Y
#     )
#     line = dcc.Graph(
#             id = f"{instate}-revenue",
#             figure = {
#                 "data" : [data],
#                 "layout" : go.Layout(
#                     xaxis={"title" : "Years"}, 
#                     yaxis={"title" : "Revenue"}
#                 )
#             }
#         )
#     return line

if __name__ == "__main__":
    from waitress import serve
    serve(app, host="0.0.0.0", port=8080)
    app.run_server()




C:\Users\A_R_COMPUTERS\Anaconda3\lib\site-packages\plotly\graph_objs\_deprecations.py:385: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [22/Sep/2020 12:58:58] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2020 12:58:58] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2020 12:58:58] "GET /_favicon.ico?v=1.16.0 HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2020 12:59:01] "GET /_dash-layout HTTP/1.1" 200 -
C:\Users\A_R_COMPUTERS\Anaconda3\lib\site-packages\ipykernel_launcher.py:235: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

127.0.0.1 - - [22/Sep/2020 12:59:01] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2020 12:59:01] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2020 12:59:01] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2020 12:59:01] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0